## Homework: Multilingual Embedding-based Machine Translation (7 points)

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully). 

For our system we choose two kindred Slavic languages: Ukrainian and Russian. 

### Feel the difference!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [2]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Download embeddings here:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

Load embeddings for ukrainian and russian.

In [3]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec/cc.uk.300.vec")

In [5]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec/cc.ru.300.vec")

In [6]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153319358826),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222576022148132),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930035829544067),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796871185303),
 ('февраль', 0.8401416540145874)]

In [7]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 1.0),
 ('липень', 0.9096439480781555),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431221961975)]

In [8]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435287714004517),
 ('конструктивность', 0.23293082416057587),
 ('офор', 0.23256802558898926),
 ('deteydlya', 0.23031717538833618),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608885169029236),
 ('подход', 0.2230587601661682),
 ('иболее', 0.22003725171089172),
 ('2015Александр', 0.21872764825820923),
 ('конструктивен', 0.21796566247940063)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [31]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    # fixed UTF-8 encoding
    with open(filename, "r", encoding="utf-8") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [32]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [33]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

In [48]:
print('Example of embedding vector in X training dataset:', X_train[0][:10])
print('Example of embedding vector in Y training dataset:', Y_train[0][:10])
print('Pairs of words: russian-ukranian:', uk_ru_train[:10])

Example of embedding vector in X training dataset: [ 0.1638  0.0508  0.0287 -0.1338  0.0469 -0.0179 -0.0088  0.0498 -0.0667
  0.1047]
Example of embedding vector in Y training dataset: [-0.0138 -0.0218 -0.0034  0.0598  0.0081  0.0153  0.0111  0.0077 -0.0113
  0.0104]
Pairs of words: russian-ukranian: [('iснує', 'существует'), ('абияк', 'как-нибудь'), ('або', 'или'), ('або', 'ли'), ('абсолютно', 'совершенно'), ('абсолютно', 'совсем'), ('автомобіль', 'автомобиль'), ('автомобіль', 'вагон'), ('агов', 'эй'), ('аґрус', 'крыжовник')]


## Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We make same bad things with source embedding space. Our Procrustean bed is target embedding space.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

But wait...$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [60]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics 

# YOUR CODE HERE
reg = LinearRegression().fit(X_train, Y_train)
slope = reg.coef_
intercept = reg.intercept_

print('Coefficients of regression:', len(slope))
print('Intercepts of regression:', len(intercept))

Coefficients of regression: 300
Intercepts of regression: 300


### Addition: Regression as an Orthogonal projection
*Source: https://nbviewer.jupyter.org/github/MakarovArtyom/Mathematics-for-Machine-Learning/blob/master/PCA/4.%20orthogonal%20projections%2C%20eigenfaces%20and%20least%20squares.ipynb*

**Key idea**: Represent $W$ as a likelihood-estimator $\theta$ that can be found using closed-form solution.

If we collect the dataset into a $(N,D)$ data matrix $\boldsymbol X$, we can write down our model like this:

$$
\begin{bmatrix}
\boldsymbol{x}_1^T \\
\vdots \\
\boldsymbol{x}_N^T
\end{bmatrix} \boldsymbol{\theta} = \begin{bmatrix}
y_1 \\
\vdots \\
y_2
\end{bmatrix},
$$

i.e.,

$$
\boldsymbol X\boldsymbol{\theta} = \boldsymbol{y}.
$$

Note that the data points are the *rows* of the data matrix, i.e., every column is a dimension of the data.

Our goal is to find the best $\boldsymbol\theta$ such that we minimize the following objective (least square).

$$
\begin{eqnarray}
& \sum^n_{i=1}{\lVert \bar{y_i} - y_i \rVert^2} \\
&= \sum^n_{i=1}{\lVert \boldsymbol \theta^T\boldsymbol{x}_i - y_i \rVert^2} \\
&= (\boldsymbol X\boldsymbol {\theta} - \boldsymbol y)^T(\boldsymbol X\boldsymbol {\theta} - \boldsymbol y).
\end{eqnarray}
$$

If we set the gradient of the above objective to $\boldsymbol 0$, we have
$$
\begin{eqnarray}
\nabla_\theta(\boldsymbol X\boldsymbol {\theta} - \boldsymbol y)^T(\boldsymbol X\boldsymbol {\theta} - \boldsymbol y) &=& \boldsymbol 0 \\
\nabla_\theta(\boldsymbol {\theta}^T\boldsymbol X^T - \boldsymbol y^T)(\boldsymbol X\boldsymbol {\theta} - \boldsymbol y) &=& \boldsymbol 0 \\
\nabla_\theta(\boldsymbol {\theta}^T\boldsymbol X^T\boldsymbol X\boldsymbol {\theta} - \boldsymbol y^T\boldsymbol X\boldsymbol \theta - \boldsymbol \theta^T\boldsymbol X^T\boldsymbol y + \boldsymbol y^T\boldsymbol y ) &=& \boldsymbol 0 \\
2\boldsymbol X^T\boldsymbol X\theta - 2\boldsymbol X^T\boldsymbol y &=& \boldsymbol 0 \\
\boldsymbol X^T\boldsymbol X\boldsymbol \theta &=& \boldsymbol X^T\boldsymbol y.
\end{eqnarray}
$$

The solution that gives zero gradient solves (which we call the maximum likelihood estimator) the following equation:

$$\boldsymbol X^T\boldsymbol X\boldsymbol \theta = \boldsymbol X^T\boldsymbol y.$$

_This is exactly the same as the normal equation we have for projections_.

This means that if we solve for $\boldsymbol X^T\boldsymbol X\boldsymbol \theta = \boldsymbol X^T\boldsymbol y.$ we would find the best $\boldsymbol \theta = (\boldsymbol X^T\boldsymbol X)^{-1}\boldsymbol X^T\boldsymbol y$, i.e. the $\boldsymbol \theta$ which minimizes our objective.

In [61]:
# maximum likelihood estimator
theta_hat = np.linalg.solve(X_train.T @ X_train, X_train.T @ Y_train)

In [106]:
print('Size of Maximum-likelihood estimator:', theta_hat.shape)

Size of Maximum-likelihood estimator: (300, 300)


Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [107]:
august = reg.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541593551635742),
 ('июнь', 0.8411964178085327),
 ('март', 0.8397400975227356),
 ('сентябрь', 0.8359216451644897),
 ('февраль', 0.8328748941421509),
 ('октябрь', 0.8311806321144104),
 ('ноябрь', 0.8278146982192993),
 ('июль', 0.8236351013183594),
 ('август', 0.8120613098144531),
 ('декабрь', 0.8038000464439392)]

In [250]:
print('Size of predicted embeddings:',august[0].shape)

Size of predicted embeddings: (300,)


We can see that neighbourhood of this embedding cosists of different months, but *right variant is on the ninth place*.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [272]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space (prediction)
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    #for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
    # check if word is in top N list
    for idx, pair in enumerate(pairs):
        vector_embed = mapped_vectors[idx].reshape(1,-1)         
        top_10 = np.array(ru_emb.most_similar(vector_embed)) # outputs top 10 neighbours
        top_pairs= top_10[:topn]
        ru_item = pair[1]
        # sum will be either 0 or 1 since all outputs of predictive model are unique
        sum_topN = sum(top_pairs[:,0]==ru_item)
        # update number of matches
        num_matches += sum_topN
        
    precision_val = num_matches / len(pairs)
    return precision_val


In [273]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [275]:
assert precision(uk_ru_test, X_test) == 0.0 # in this case we feed ukranian embeddings (X_test) and compare to russian words. The precision is 0
assert precision(uk_ru_test, Y_test) == 1.0 # here we compare russian embeddings (Y_test since target it russian translation) and russian words

In [277]:
precision_top1 = precision(uk_ru_test, reg.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, reg.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.713  # initially 0.813

In [279]:
print('Precision top-1', precision_top1)
print('Precision top-5', precision_top5)

Precision top-1 0.6356589147286822
Precision top-5 0.8113695090439277


## Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [284]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOU CODE HERE
    return np.linalg.pinv(X_train).dot(Y_train)

In [285]:
W = learn_transform(X_train, Y_train)

In [291]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8541286587715149),
 ('июнь', 0.8411203026771545),
 ('март', 0.8396995067596436),
 ('сентябрь', 0.8359869122505188),
 ('февраль', 0.8329298496246338),
 ('октябрь', 0.8311846256256104),
 ('ноябрь', 0.8278923630714417),
 ('июль', 0.8234529495239258),
 ('август', 0.8120501041412354),
 ('декабрь', 0.8039003610610962)]

In [297]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.622 
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.811 

In [298]:
print('Precision top-1', precision(uk_ru_test, np.matmul(X_test, W)))
print('Precision top-5', precision(uk_ru_test, np.matmul(X_test, W), 5) )

Precision top-1 0.6356589147286822
Precision top-5 0.813953488372093


In [299]:
print('Precision top-1 (theta)', precision(uk_ru_test, np.matmul(X_test, theta_hat)))
print('Precision top-5 (theta)', precision(uk_ru_test, np.matmul(X_test, theta_hat), 5) )

Precision top-1 0.6356589147286822
Precision top-5 0.813953488372093


## UK-RU Translator

Now we are ready to make simple word-based translator: for earch word in source language in shared embedding space we find the nearest in target language.


In [390]:
with open("fairy_tale.txt", "r", encoding="utf-8") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [397]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE HERE
    sentence_new = []
    sent = sentence.split(" ")
    for word in sent:
        ukr_embed = uk_emb[word].reshape(1, -1)
        top_word = ru_emb.most_similar(ukr_embed.dot(W))[0][0]
        sentence_new.append(top_word)
    sentence_rus = ' '.join(sentence_new)
    return sentence_rus

In [398]:
#assert translate(".") == "."
#assert translate("1 , 3") == "1 , 3"
#assert translate("кіт зловив мишу") == "кот поймал мышку"

In [399]:
translate("кіт зловив мишу")

'кот поймал мышь'

In [411]:
translate("лисичка - сестричка і вовк")

'лисичка – девочка и волк'

In [409]:
sent = uk_sentences[0]
sent

'лисичка - сестричка і вовк - панібрат'

for sentence in uk_sentences[50:51]:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

Not so bad, right? We can easily improve translation using language model and not one but several nearest neighbours in shared embedding space. But next time.

## Would you like to learn more?

### Articles:
* [Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168)  - entry point for multilingual embedding studies by Tomas Mikolov (the author of W2V)
* [Offline bilingual word vectors, orthogonal transformations and the inverted softmax](https://arxiv.org/pdf/1702.03859) - orthogonal transform for unsupervised MT
* [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087)
* [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745)
* [Unsupervised Alignment of Embeddings with Wasserstein Procrustes](https://arxiv.org/pdf/1805.11222)

### Repos (with ready-to-use multilingual embeddings):
* https://github.com/facebookresearch/MUSE

* https://github.com/Babylonpartners/fastText_multilingual -